In [1]:
# History Data (history.json and diff.json) from: https://github.com/mumuy/data_location/

In [12]:
import pandas as pd
import json

In [3]:
df_2020 = pd.read_csv('2020ChinaCountyTable.csv')
df_2020

,CODE2020,COUNTY_CH,COUNTY_EN,DCODE,CITY_CH,CITY_EN,PCODE,PROV_CH,PROV_EN
0,110101,东城区,DongChengQu,1101,北京市辖区,BeiJingShiXiaQu,11,北京市,BeiJinShi
1,110102,西城区,XiChengQu,1101,北京市辖区,BeiJingShiXiaQu,11,北京市,BeiJinShi
2,110105,朝阳区,ChaoYangQu,1101,北京市辖区,BeiJingShiXiaQu,11,北京市,BeiJinShi
3,110106,丰台区,FengTaiQu,1101,北京市辖区,BeiJingShiXiaQu,11,北京市,BeiJinShi
4,110107,石景山区,ShiJingShanQu,1101,北京市辖区,BeiJingShiXiaQu,11,北京市,BeiJinShi
...,...,...,...,...,...,...,...,...,...
2850,659010,胡杨河市,HuYangHeShi,6590,自治区直辖县级行政单位,ZiZhiQuZhiXiaXianJiXingZhengDanWei,65,新疆维吾尔自治区,XinJiangWeiWuErZuZiZhiQu
2851,232761,加格达奇区,JiaGeDaQiQu,2327,大兴安岭地区（加格达奇）,DaXingAnLingDiQu（JiaGeDaQi）\r\n,23,黑龙江省,HeiLongJiangSheng
2852,232763,新林区,XinLinQu,2327,大兴安岭地区（加格达奇）,DaXingAnLingDiQu（JiaGeDaQi）,23,黑龙江省,HeiLongJiangSheng
2853,232764,呼中区,Huzhongqu,2327,大兴安岭地区（加格达奇）,DaXingAnLingDiQu（JiaGeDaQi）,23,黑龙江省,HeiLongJiangSheng


In [4]:
with open('diff.json', 'r') as f:
    diff = json.load(f)

with open('history.json', 'r') as f:
    history = json.load(f)

In [5]:
def convert_id(id):
    id = str(id).rstrip('0')
    if len(id) == 5 or len(id) == 3 or len(id) == 1:
        id = id + '0'
    id = int(id)
    return id

In [6]:
# dict to dataframe
df_diff = pd.DataFrame(columns=['history_id', 'current_id_list'])
df_diff['history_id'] = diff.keys()
df_diff['current_id_list'] = diff.values()

# df_diff = df_diff[df_diff['current_id_list'].map(len) > 0]

# only deal with the cases where the length of current_id_list is 1
df_diff = df_diff[df_diff['current_id_list'].map(len) == 1]
df_diff['current_id'] = df_diff['current_id_list'].map(lambda x: x[0])

df_diff['history_name'] = df_diff['history_id'].map(lambda x: history[x] if x in history else '')
df_diff['current_name'] = df_diff['current_id'].map(lambda x: history[x] if x in history else '')

df_diff['history_id'] = df_diff['history_id'].map(lambda x: convert_id(x))
df_diff['current_id'] = df_diff['current_id'].map(lambda x: convert_id(x))
df_diff = df_diff[~df_diff['history_id'].isin(df_2020['CODE2020'])]

In [7]:
def filter_diff(row):
    current_name = row['current_name'].replace('市', '').replace('区', '').replace('县', '').replace('自治州', '').replace('自治县', '').replace('自治旗', '').replace('自治区', '').replace('特别行政区', '')
    history_name = row['history_name'].replace('市', '').replace('区', '').replace('县', '').replace('自治州', '').replace('自治县', '').replace('自治旗', '').replace('自治区', '').replace('特别行政区', '')
    if history_name == current_name:
        return False
    else:
        return True

df_diff = df_diff[df_diff.apply(filter_diff, axis=1)]


In [8]:
def detect_level(row):
    if len(str(row['current_id'])) == 6:
        return 'County'
    elif len(str(row['current_id'])) == 4:
        return 'Prefecture'
    elif len(str(row['current_id'])) == 2:
        return 'Province'
    else:
        return 'Error'

df_diff['level'] = df_diff.apply(detect_level, axis=1)
df_diff

,history_id,current_id_list,current_id,history_name,current_name,level
1,110103,[110101],110101,崇文区,东城区,County
2,110104,[110102],110102,宣武区,西城区,County
3,110110,[110111],110111,燕山区,房山区,County
6,110203,[110112],110112,通县,通州区,County
15,110223,[110112],110112,通县,通州区,County
...,...,...,...,...,...,...
3937,652303,[650109],650109,米泉市,米东区,County
3939,652322,[650109],650109,米泉市,米东区,County
3958,652524,[654281],654281,沙湾县,,County
3977,6541,[654000],6540,伊犁地区,伊犁哈萨克自治州,Prefecture


In [9]:
df_2020[df_2020['CODE2020'] == 659002].iloc[0].to_dict()


{'CODE2020': 659002,
 'COUNTY_CH': '阿拉尔市',
 'COUNTY_EN': 'ALaErShi',
 'DCODE': 6590,
 'CITY_CH': '自治区直辖县级行政单位',
 'CITY_EN': 'ZiZhiQuZhiXiaXianJiXingZhengDanWei',
 'PCODE': 65,
 'PROV_CH': '新疆维吾尔自治区',
 'PROV_EN': 'XinJiangWeiWuErZuZiZhiQu'}

In [10]:
df_2020['NOTE'] = None
for id, row in df_diff.iterrows():
    level = row['level']
    id = row['current_id']
    name = row['history_name']
    if level == 'County':
        try:
            new_row = df_2020[df_2020['CODE2020'] == id].iloc[0].to_dict()
            new_row['COUNTY_CH'] = name
            new_row['COUNTY_EN'] = 'No Data'
            new_row['NOTE'] = 'History Data'
            df_2020.loc[len(df_2020)] = new_row
        except:
            pass
    elif level == 'Prefecture':
        try:
            new_row = df_2020[df_2020['DCODE'] == id].iloc[0].to_dict()
            new_row['CODE2020'] = 0
            new_row['COUNTY_CH'] = 'No Data'
            new_row['COUNTY_EN'] = 'No Data'
            new_row['NOTE'] = 'History Data'
            df_2020.loc[len(df_2020)] = new_row
        except:
            pass


In [11]:
df_2020.to_csv('ModernChinaCountyTable.csv', index=False, encoding='utf-8-sig')